<a href="https://colab.research.google.com/github/SeoYeonJu4382/bioinfo_1_project_yjseo/blob/main/Final_Free_Proj_2022_24764.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# < 생물정보학 및 실습 1 - Final Free Project >
##2022-24764 서연주
### 1. Thema : Random forest를 이용하여 LIN28A binding motif predictor 만들기

### 2. Tools : Running from Mac and using python and R

###3. Object :

CLIP-seq로 LIN28A-RNA complex를 구현 함으로써 LIN28A의 binding motif에 대한 특징을 파악할 수 있다.
LIN28A-bound sequence들로 기계학습으로 예측 모델을 만들고 평가해 봄으로써 key sequence들을 파악할 수 있다.
LIN28A binding motif predictor를 Random forest 모델을 적용하여 생성하고 학습시킬 수 있다.
Random forest 모델링을 진행해 봄으로써 모델에 대한 이해와 방법론에 대한 원리에 대한 이해를 넓힐 수 있다.
### Final Free Project용 환경 셋업

```
생물정보학 및 실습 1

이 노트북은 Creative Commons BY-SA 조건으로 재사용 가능합니다.
장혜식 <hyeshik@snu.ac.kr>, 서울대학교 생명과학부
```



## Google Drive 마운트

데이터 파일들을 불러오고 저장하기 위해서 Google Drive를 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 실습용 Bioconda 환경 설치하기


In [2]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.bashrc.biolab
./root/.tmux.conf
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip3
./root/.bin.priority/pip2
./root/.profile
./root/.vimrc
./root/.condarc
--2022-06-03 08:58:14--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M   104MB/s    in 0.7s    

2022-06-03 08:58:15 (104 MB/s) - ‘miniconda3.sh’ save

설정이 끝났으면, 이제 콘솔을 열어서 터미널로 접속할 수 있는 통로를 열어줍니다.

## Google Drive에 시퀀싱 데이터 다운로드: 


In [ ]:
!wget -O - --no-check-certificate https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar | tar -C /content/drive/MyDrive -xf -

--2022-04-29 06:03:56--  https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar
Resolving hyeshik.qbio.io (hyeshik.qbio.io)... 147.47.203.129
Connecting to hyeshik.qbio.io (hyeshik.qbio.io)|147.47.203.129|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6424668160 (6.0G) [application/octet-stream]
Saving to: ‘STDOUT’

-                    16%[==>                 ]   1.00G  2.49MB/s    in 6m 3s   

2022-04-29 06:10:00 (2.84 MB/s) - Connection closed at byte 1078050517. Retrying.

--2022-04-29 06:10:01--  (try: 2)  https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar
Connecting to hyeshik.qbio.io (hyeshik.qbio.io)|147.47.203.129|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 206 Partial Content
Length: 6424668160 (6.0G), 5346617643 (5.0G) remaining [application/octet-stream]
Saving to: ‘STDOUT’

-                    33%[+++==>              ]   2.01G  2.63MB

받은 데이터의 MD5 체크섬을 확인합니다. 한참 걸립니다. 각 파일들의 원래 체크섬은 다음과 같습니다. 한 번 비교해 보세요.

```
140aaf30bcb9276cc716f8699f04ddd6  CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  CLIP-35L33G.bam.bai
328883a73d507eafbf5b60bd6b906201  RNA-control.bam
02073818e2f398a73c3b76e5169de1ca  RNA-control.bam.bai
b09550d09d6c2a4ce27f0226f426fdb1  RNA-siLin28a.bam
fef112c727244060ea62d3f2564a07f6  RNA-siLin28a.bam.bai
28bbd0c47d725669340c784f1b772c01  RNA-siLuc.bam
43590fdc4d81905c0432e0d1cb8cfd5b  RNA-siLuc.bam.bai
5c08a9297307bc83259e658c4474f0cc  RPF-siLin28a.bam
a1bb3e29be412dfd7fd8d16b1b1acc4c  RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707  RPF-siLuc.bam
dc24f69e8f571fc8be30f28ce5b84fcd  RPF-siLuc.bam.bai
```

In [ ]:
!md5sum drive/MyDrive/binfo1-datapack1/*

140aaf30bcb9276cc716f8699f04ddd6  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam.bai
d54d428f5872030c4ffc43058709b896  drive/MyDrive/binfo1-datapack1/gencode.gtf
794eb8d273987a094dcc013afa2a1956  drive/MyDrive/binfo1-datapack1/gencode.gtf.gz
328883a73d507eafbf5b60bd6b906201  drive/MyDrive/binfo1-datapack1/RNA-control.bam
02073818e2f398a73c3b76e5169de1ca  drive/MyDrive/binfo1-datapack1/RNA-control.bam.bai
b09550d09d6c2a4ce27f0226f426fdb1  drive/MyDrive/binfo1-datapack1/RNA-siLin28a.bam
fef112c727244060ea62d3f2564a07f6  drive/MyDrive/binfo1-datapack1/RNA-siLin28a.bam.bai
28bbd0c47d725669340c784f1b772c01  drive/MyDrive/binfo1-datapack1/RNA-siLuc.bam
5c08a9297307bc83259e658c4474f0cc  drive/MyDrive/binfo1-datapack1/RPF-siLin28a.bam
a1bb3e29be412dfd7fd8d16b1b1acc4c  drive/MyDrive/binfo1-datapack1/RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707  drive/MyDrive/binfo1-datapack1/RPF-siLuc.bam
dc24f69e8f571fc8b

실습에 필요한 프로그램들 몇 가지를 추가로 설치 및 디렉토리 설정

In [ ]:
!conda install -y subread

/bin/bash: conda: command not found


In [3]:
%cd /content/drive/MyDrive/binfo1-datapack1/

/content/drive/MyDrive/binfo1-datapack1


## 01. reference data 불러오기

In [5]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 5.2 MB/s 


In [6]:
from Bio import SeqIO

for seq_record in SeqIO.parse("GRCm39.primary_assembly.genome.fa", "fasta"):
  print(seq_record.id)
  print(repr(seq_record.seq))
  print(len(seq_record))

chr1
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
195154279
chr2
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
181755017
chr3
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
159745316
chr4
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
156860686
chr5
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
151758149
chr6
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
149588044
chr7
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
144995196
chr8
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
130127694
chr9
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
124359700
chr10
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
130530862
chr11
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
121973369
chr12
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
120092757
c

In [ ]:
for seq_record in SeqIO.parse("GRCm39.primary_assembly.genome.fa", "fasta"):
  if seq_record.id =='chr19':
    seq = seq_record.seq
    print(repr(seq_record.seq))

Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')


## 02. Pileup data 생성

In [7]:
!samtools view -b -o CLIP-chr19.bam CLIP-35L33G.bam chr19
!samtools view CLIP-chr19.bam | wc -l

823068


In [9]:
!samtools view CLIP-chr19.bam | head -5

SRR458758.403627	256	chr19	3051244	1	23M	*	0	0	AAGAGCAGGTGGATATTCTTTGG	IIIIIIGIIFIIGIGGIIIIHII	MD:Z:23	NH:i:7	HI:i:7	NM:i:0	SM:i:1	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.1709140	256	chr19	3051244	1	23M	*	0	0	AAGAGCAGGTGGATATTCTTTGG	B;;675BB??DD:B7@3>4@/0@	MD:Z:23	NH:i:7	HI:i:7	NM:i:0	SM:i:1	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.7712889	256	chr19	3051244	1	23M	*	0	0	AAGAGCAGGTGGATATTCTTTGG	@GGGB>DCBCA8F>FCCGG@FGF	MD:Z:23	NH:i:7	HI:i:7	NM:i:0	SM:i:1	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23388448	256	chr19	3051244	1	23M	*	0	0	AAGAGCAGGTGGATATTCTTTGG	IIIEIIIIHIIIIIIIIIIHIIE	MD:Z:23	NH:i:7	HI:i:7	NM:i:0	SM:i:1	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.25211653	256	chr19	3051244	1	23M	*	0	0	AAGAGCAGGTGGATATTCTTTGG	GE?D?FEFFADGEDGDFF>G@>B	MD:Z:23	NH:i:7	HI:i:7	NM:i:0	SM:i:1	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-


In [8]:
!samtools sort -o CLIP-chr19.sorted.bam CLIP-chr19.bam
!samtools index CLIP-chr19.sorted.bam

In [10]:
!samtools mpileup CLIP-chr19.bam > CLIP-chr19.pileup
!wc -l CLIP-chr19.pileup

[mpileup] 1 samples in 1 input files
23000226 CLIP-chr19.pileup


In [11]:
!head CLIP-chr19.pileup

chr19	3059904	N	17	^IA^GA^IA^IA^IA^HA^IA^GA^HA^IA^IA^IA^GA^IA^HA^GA^GA	IGIGIHGGHIFFGIHGD
chr19	3059905	N	17	GGGGGGGGGGGGGGGGG	I@BIGHGGHIIIGIGGG
chr19	3059906	N	17	AAAAAAAAAAAAAAAAA	I>IIEHGGHIIIGIHGD
chr19	3059907	N	17	AAAAAAAAAAAAAAAAA	IGIIHHGEHIIIGIHGF
chr19	3059908	N	17	TTTTTTTTTTTTTTTTT	IGIIIHGEHIIIGIHDG
chr19	3059909	N	17	TTTTTTTTTTTTTTTTT	IAGIIHHGHIIIGIHGD
chr19	3059910	N	17	AAAAAAAAAAAAAAAAA	IEIFIGI<HHIHGIB@G
chr19	3059911	N	17	AAAAAAAAAAAAAAAAA	IBHIIHIGHHIIGIDGG
chr19	3059912	N	17	AAAAAAAAAAAAAAAAA	I?IIIEIGHHIIGIHGG
chr19	3059913	N	17	AAAAAAAAAAAAAAAAA	I>IGIHI8HHIIGIH@D


In [12]:
!awk '$2 { print $0; }' CLIP-chr19.pileup > CLIP-chr19-gene.pileup
!tail CLIP-chr19-gene.pileup

chr19	61316972	N	8	TTTTTTTT	DG<IEEAE
chr19	61316973	N	8	AAAAAAAA	GD9FEGCG
chr19	61316974	N	8	AAAAAAAA	EGBIGGGG
chr19	61316975	N	8	TTTTTTTT	GGBIGGBG
chr19	61316976	N	8	AAAAAAAA	EG>GGGEG
chr19	61316977	N	8	AAAAAAAA	FGBGGGFG
chr19	61316978	N	8	GGGGGGGG	EGBGGGEG
chr19	61316979	N	8	GGGGGGGG	FE:GGGFD
chr19	61316980	N	8	TTTTTTTT	ADCGDGCD
chr19	61316981	N	8	T$T$T$T$T$T$T$T$	DGBIIIGI


In [14]:
import pandas as pd

pileup = pd.read_csv('CLIP-chr19-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

,chrom,pos,_ref,count,basereads,quals
23000221,chr19,61316977,N,8,AAAAAAAA,FGBGGGFG
23000222,chr19,61316978,N,8,GGGGGGGG,EGBGGGEG
23000223,chr19,61316979,N,8,GGGGGGGG,FE:GGGFD
23000224,chr19,61316980,N,8,TTTTTTTT,ADCGDGCD
23000225,chr19,61316981,N,8,T$T$T$T$T$T$T$T$,DGBIIIGI


In [15]:
pileup_long_read = pileup[pileup['count'] >= 50]
pileup_long_read.tail()

,chrom,pos,_ref,count,basereads,quals
22998198,chr19,61293501,N,63,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,GEIIII;IBI=GI;I?IHIIIIIFG?@I;GIFHBGIHHIIGB=IHH...
22998199,chr19,61293502,N,63,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa$a...,GGIHII4I?IGGI6IDHHIIIIIFGB;I2DIBH?DIIHIIHE:IHH...
22998200,chr19,61293503,N,59,cccccccccccccccccccccccccccccccccccccccccccccc...,GGIIII;IBIGGI;HDIHIIIIIFG.AIA@IFHBGIIHGIHEBHH?...
22998201,chr19,61293504,N,58,tttttttttttttttttttttttttttttttttttttttttttttt...,GDIIHI7IBIEGI:HBIGIIIIIFGB8I@GIFHBGIIHGIFBHH=?...
22998202,chr19,61293505,N,59,t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$t$...,GGIIHI?ICIGGIBHDIGIIIIIFG:AIAGGFHCGIDHIIBEBBGD...


pileup에서 계산에 사용될 match와 substitution만 남기고 다른 tag들은 제거

In [17]:
import re
toremove = re.compile('[<>$*#^]')
pileup_long_read['matches'] = pileup_long_read['basereads'].apply(lambda x: toremove.sub('', x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


match와 substitution만 남긴 데이터 생성

In [19]:
pileup_long_read[['chrom', 'pos', 'matches']]

,chrom,pos,matches
12829,chr19,3335340,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAIAI...
12830,chr19,3335341,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
12831,chr19,3335342,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
12832,chr19,3335343,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...
12833,chr19,3335344,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...
...,...,...,...
22998198,chr19,61293501,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
22998199,chr19,61293502,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
22998200,chr19,61293503,cccccccccccccccccccccccccccccccccccccccccccccc...
22998201,chr19,61293504,tttttttttttttttttttttttttttttttttttttttttttttt...


In [20]:
pileup_long_read.to_csv('pileup_long_read.csv') # model input sequencce 생성! 

## 03. 각 position별로 base수를 세기 

In [21]:
pileup_long_read_data = pileup_long_read[['chrom', 'pos', 'matches']]
pileup_long_read_data['base'] = pileup_long_read_data['matches'].str.len()

pileup_long_read_data.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,chrom,pos,matches,base
12829,chr19,3335340,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAIAI...,59
12830,chr19,3335341,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,50
12831,chr19,3335342,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,62
12832,chr19,3335343,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,56
12833,chr19,3335344,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,56
12834,chr19,3335345,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,68
12835,chr19,3335346,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,62
12836,chr19,3335347,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,62
12837,chr19,3335348,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,110
12838,chr19,3335349,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,86


# 04. 각 position별로 Shannon entropy를 계산

In [22]:
# Shannon entropy 계산 함수

import collections
import math
 
 
def estimate_shannon_entropy(dna_sequence):
    m = len(dna_sequence)
    bases = collections.Counter([tmp_base for tmp_base in dna_sequence])
 
    shannon_entropy_value = 0
    for base in bases:
        # number of residues
        n_i = bases[base]
        # n_i (# residues type i) / M (# residues in column)
        p_i = n_i / float(m)
        entropy_i = p_i * (math.log(p_i, 2))
        shannon_entropy_value += entropy_i
 
    return shannon_entropy_value * -1

In [24]:
pileup_long_read_data['entropy'] = pileup_long_read_data['matches'].map(lambda x : estimate_shannon_entropy(x))
pileup_long_read_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,chrom,pos,matches,base,entropy
12829,chr19,3335340,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAIAI...,59,0.802938
12830,chr19,3335341,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,50,-0.000000
12831,chr19,3335342,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,62,0.599894
12832,chr19,3335343,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,56,-0.000000
12833,chr19,3335344,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,56,-0.000000
...,...,...,...,...,...
22998198,chr19,61293501,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,63,-0.000000
22998199,chr19,61293502,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,63,-0.000000
22998200,chr19,61293503,cccccccccccccccccccccccccccccccccccccccccccccc...,59,-0.000000
22998201,chr19,61293504,tttttttttttttttttttttttttttttttttttttttttttttt...,58,-0.000000


In [26]:
pileup_long_read_data['end'] = pileup_long_read_data['pos'] +  pileup_long_read['basereads'].apply(len)
pileup_long_read_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,chrom,pos,matches,base,entropy,end
12829,chr19,3335340,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAIAIAI...,59,0.802938,3335411
12830,chr19,3335341,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,50,-0.000000,3335394
12831,chr19,3335342,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,62,0.599894,3335413
12832,chr19,3335343,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,56,-0.000000,3335402
12833,chr19,3335344,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,56,-0.000000,3335403
...,...,...,...,...,...,...
22998198,chr19,61293501,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,63,-0.000000,61293564
22998199,chr19,61293502,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,63,-0.000000,61293569
22998200,chr19,61293503,cccccccccccccccccccccccccccccccccccccccccccccc...,59,-0.000000,61293562
22998201,chr19,61293504,tttttttttttttttttttttttttttttttttttttttttttttt...,58,-0.000000,61293562


In [27]:
# 계산한 Shannon entropy를 bedgraph format으로 출력해보기

chr19_bedgraph_pileup_data = pileup_long_read_data[['chrom','pos','end','entropy']]
chr19_bedgraph_pileup_data.columns = ['chromosome','start','end', 'entropy' ]

chr19_bedgraph_pileup_data

,chromosome,start,end,entropy
12829,chr19,3335340,3335411,0.802938
12830,chr19,3335341,3335394,-0.000000
12831,chr19,3335342,3335413,0.599894
12832,chr19,3335343,3335402,-0.000000
12833,chr19,3335344,3335403,-0.000000
...,...,...,...,...
22998198,chr19,61293501,61293564,-0.000000
22998199,chr19,61293502,61293569,-0.000000
22998200,chr19,61293503,61293562,-0.000000
22998201,chr19,61293504,61293562,-0.000000


In [28]:
chr19_bedgraph_pileup_data.to_csv("chr19_bedgraph.bed",header=None, index=False, sep='\t')